In [1]:
import os
import re
from copy import deepcopy
from pathlib import Path

import cheetah.bmad

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [3]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [5]:
lines = cheetah.bmad.read_clean_lines(resolved_lattice_file_path)
lines[:20]

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0',
 'sethxrss = 0',
 'setsxrss = 0',
 'setcbxfel = 0',
 'setpepx = 0',
 'intgsx = 30.0']

In [6]:
merged_lines = cheetah.bmad.merge_ampersand_continued_lines(lines)
len(lines), len(merged_lines)

(14409, 14405)

In [50]:
property_assignment_pattern = r"[a-z0-9]+\[[a-z_]+\]\s*=.*"
variable_assignment_pattern = r"[a-z0-9_]+\s*=.*"
element_pattern = r"[a-z0-9_]+\:.*"
line_pattern = r"[a-z0-9_]+\s*\:\s*line\s*=\s*\((\s*[a-z0-9_]+\s*\,)*\s*[a-z0-9_]+\s*\)"
incomplete_line_pattern_begin = r"[a-z0-9_]+\s*\:\s*line\s*=\s*\((\s*[a-z0-9_]+\s*\,)*"
incomplete_line_pattern_middle = r"(\s*[a-z0-9_]+\s*\,)*"
incomplete_line_pattern_end = r"(\s*[a-z0-9_]+\s*\,)*\s*[a-z0-9_]+\s*\)"
coninued_expression_pattern = (
    r"[\+\-\*\/]?[a-z0-9_\[\]]+(\s*[\+\-\*\/]\s*[a-z0-9_\[\]]+)+(\s*[\+\-\*\/]\s*\&)?"
)

num_successful = 0
num_property_assignment = 0
num_variable_assignment = 0
num_element = 0
num_line = 0
num_incomplete_line_begin = 0
num_incomplete_line_middle = 0
num_incomplete_line_end = 0
num_continued_expression = 0
for line in lines:
    if re.fullmatch(property_assignment_pattern, line):
        num_successful += 1
        num_property_assignment += 1
    elif re.fullmatch(variable_assignment_pattern, line):
        num_successful += 1
        num_variable_assignment += 1
    elif re.fullmatch(element_pattern, line):
        num_successful += 1
        num_element += 1
    elif re.fullmatch(line_pattern, line):
        num_successful += 1
        num_line += 1
    elif re.fullmatch(incomplete_line_pattern_begin, line):
        num_successful += 1
        num_incomplete_line_begin += 1
    elif re.fullmatch(incomplete_line_pattern_middle, line):
        num_successful += 1
        num_incomplete_line_middle += 1
    elif re.fullmatch(incomplete_line_pattern_end, line):
        num_successful += 1
        num_incomplete_line_end += 1
    elif re.fullmatch(coninued_expression_pattern, line):
        num_successful += 1
        num_continued_expression += 1
    else:
        print(line)
        break

print("")
print("######################################")
print(f"num_successful: {num_successful} / {len(lines)}")
print("--------------------------------------")
print(f"{num_property_assignment = }")
print(f"{num_variable_assignment = }")
print(f"{num_element = }")
print(f"{num_line = }")
print(f"{num_incomplete_line_begin = }")
print(f"{num_incomplete_line_middle = }")
print(f"{num_incomplete_line_end = }")
print(f"{num_continued_expression = }")
print("######################################")

dbsy52d[l]

######################################
num_successful: 3658 / 14409
--------------------------------------
num_property_assignment = 210
num_variable_assignment = 1280
num_element = 2024
num_line = 49
num_incomplete_line_begin = 24
num_incomplete_line_middle = 44
num_incomplete_line_end = 24
num_continued_expression = 3
######################################


In [51]:
line

'dbsy52d[l]'

In [52]:
re.match(coninued_expression_pattern, line).group(0)

AttributeError: 'NoneType' object has no attribute 'group'